In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image
import ast
from IPython.display import display

In [2]:
# # Install CocoAPI tool
# %cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI

# !make
# !make install
# !python setup.py install

In [3]:
# import pycocotools

In [4]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

/kaggle/input/norfair031py3
Looking in links: ./
Processing ./commonmark-0.9.1-py2.py3-none-any.whl
Processing ./rich-9.13.0-py3-none-any.whl
/kaggle/working/tmp/filterpy-1.4.5
Processing /kaggle/working/tmp/filterpy-1.4.5
  Preparing metadata (setup.py) ... - done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=de0a7f4ed4a20c4ab19e436b7f98c3d5fe64aac8b5a8db71eea6a1d8031ae511
  Stored in directory: /root/.cache/pip/wheels/0e/ba/6d/20557ee403bc0c849d9fe3b358ded9918d7d2b0c5b0253fc99
Successfully built filterpy
/kaggle/input/norfair031py3
Looking in links: ./
Processing ./norfair-0.3.1-py3-none-any.whl


In [5]:
cd /kaggle/working

/kaggle/working


In [6]:
CONF      = 0.50
IOU       = 0.50
IMG_SIZE  = 7000
AUGMENT   = True
# CKPT_PATH = '../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt'
# CKPT_PATH = '../input/yolov5-1920-4/best.pt'#public best weight
CKPT_PATH = '../input/yolov5s6/best.pt'#public best weight
# CKPT_PATH = '../input/yolov5s6dim3000fold1numfold10/best.pt' #StUr team weight

In [7]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [8]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [9]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
display(df.head(2))

  0%|          | 0/23501 [00:00<?, ?it/s]

  0%|          | 0/23501 [00:00<?, ?it/s]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,image_path
0,0,40258,0,0,0-0,[],/kaggle/input/tensorflow-great-barrier-reef//t...
1,0,40258,1,1,0-1,[],/kaggle/input/tensorflow-great-barrier-reef//t...


In [10]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

  0%|          | 0/23501 [00:00<?, ?it/s]

No BBox: 79.07% | With BBox: 20.93%


In [11]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [12]:
def load_model(ckpt_path, conf=0.01, iou=0.50):
    model = torch.hub.load('../input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [13]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = []
    detects = []
    if results.pandas().xyxy[0].shape[0] == 0:
        return [],[]
    else:
        for idx, row in results.pandas().xyxy[0].iterrows():
            if row.confidence > 0.01:
                detects.append([int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax), row.confidence])
                preds.append([int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin)])
                #     bboxes  = preds[['xmin','ymin','xmax','ymax', 'confidence']].values.astype(int)
#         bboxes  = voc2coco(bboxes,height,width).astype(int)
#         confs   = preds.confidence.values
        return detects, preds
        
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((400, 200))

%matplotlib inline
def compare_img(img, tracked_img, bboxes, tracked_bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    tracked_names  = ['starfish']*len(tracked_bboxes)
    labels = [0]*len(bboxes)
    tracked_labels = [0]*len(tracked_bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    tracked_img = draw_bboxes(img = tracked_img,
                           bboxes = tracked_bboxes, 
                           classes =tracked_names,
                           class_ids = tracked_labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    plt.figure(figsize=(40,30))
    fig, axs = plt.subplots(1,2)
    plt.gca().set_title('detects by yolov5')
    axs[0].imshow(img)
    plt.gca().set_title('tracked objects by bytetracker')
    axs[1].imshow(tracked_img)
    fig.set_size_inches(38.5, 29.5)
    plt.show()
#     plt.show()

    return Image.fromarray(img).resize((800, 400))


In [14]:
##############################################################
#                      Tracking helpers                      #
##############################################################
import cv2 as cv
import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

def calculate_homography(image, next_image, min_matches=50, topk=100, alpha=0.3):
#     sift = cv2.SIFT_create()
    # AKAZE作成
    akaze= cv2.AKAZE_create()

#     # find the keypoints and descriptors with SIFT
#     kp11, des11 = sift.detectAndCompute(image, None)
#     kp21, des21 = sift.detectAndCompute(next_image, None)


    kp1, des1 = akaze.detectAndCompute(image, None)
    kp2, des2 = akaze.detectAndCompute(next_image, None)
    des1 = np.float32(des1)
    des2 = np.float32(des2)
#     print('sift{},{},{}'.format(des11,type(des11),des11.shape))
#     print('akaze{},{},{}'.format(des1,type(des1),des11.shape))
    
    min_matches = 10
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)#インデックス中の木構造を再帰的にたどる回数
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

#     # Brute-Force Matcher生成
#     bf = cv2.BFMatcher()

#     # 特徴量ベクトル同士をBrute-Force＆KNNでマッチング
#     matches = bf.knnMatch(des1, des2, k=2)

    scores = [m.distance / n.distance for m, n in matches]
    indices = np.argsort(scores)
    good = [matches[i][0] for i in indices[:topk] if scores[i] < alpha]
    if len(good) > min_matches:
        #目的画像特徴点src_pts,入力画像特徴点dst_pts
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
    else:
        print(f"Not enough matches: {len(good)}")
        M = None
        
    return M    


def transform(points, H):
    points = np.asarray(points, dtype=float).reshape(1, -1, 2)
    return cv2.perspectiveTransform(points, H)[0]


class MatchingCOTSTracker(Tracker):
    def __init__(self, *args, min_matches=50, topk=100, alpha=0.3, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.min_matches = min_matches
        self.topk = 100
        self.alpha = 0.3
        
        self.prev_frame = None
        
    def update(self, frame, frame_id, detections=None, period=1):
        self.shift_tracks(frame, frame_id)
            
        detections = to_norfair(detections, frame_id)
        tracked_objects = super().update(detections, period)
        
        result = tracked_objects
            
        self.prev_frame = frame
        
        return result
    
    def shift_tracks(self, frame, frame_id):
        if self.prev_frame is None or len(self.tracked_objects) == 0:
            return

        H = calculate_homography(
            self.prev_frame, 
            frame, 
            min_matches=self.min_matches, 
            topk=self.topk, 
            alpha=self.alpha
        )        
        if H is None:
            print(f"Can't calculate homography for frame {frame_id}")
            return
        
        for i, tobj in enumerate(self.tracked_objects):
            new_xc, new_yc = transform(tobj.estimate, H)[0]
            self.tracked_objects[i].filter.x[0] = new_xc
            self.tracked_objects[i].filter.x[1] = new_yc

In [15]:
a=df[df.sequence == 8503][1980:2050]
# a[:30].annotations

In [16]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
image_paths = df[df.sequence == 8503][1980:].image_path.tolist()
for idx, path in enumerate(image_paths):
    img = cv2.imread(path)[...,::-1]
    detects, preds = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    compare_img(img, img,preds, preds, bbox_format='coco')
    break
    if idx>0:
        break

YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 


In [17]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
# image_paths = df[100:200].image_path.tolist()
image_paths = df[df.sequence == 53708][400:].image_path.tolist()
# image_paths = df[df.sequence == 8503][2300:].image_path.tolist()

#######################################################
#                      Tracking                       #
#######################################################

# Tracker will update tracks based on detections from current frame
# Matching based on euclidean distance between bbox centers of detections 
# from current frame and tracked_objects based on previous frames
# You can check it's parameters in norfair docs
# https://github.com/tryolabs/norfair/blob/master/docs/README.md
tracker = MatchingCOTSTracker(
    distance_function=euclidean_distance, 
    distance_threshold=20,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

# Save frame_id into detection to know which tracks have no detections on current frame
frame_id = 0
#######################################################

for idx, path in enumerate(tqdm(image_paths)):
    im_for_track = cv2.imread(path)
    img = cv2.imread(path)[...,::-1]
    height, width = img.shape[:2]
    detects, preds = predict(model, img, size=IMG_SIZE, augment=AUGMENT)

#     height, width = img.shape[0], img.shape[1]
#     r = model(img, size=IMG_SIZE, augment=False)
#     detects = []
    tracked_img = img
    tracked = []
    if detects is not None:
#         for idx, row in r.pandas().xyxy[0].iterrows():
#             if row.confidence > 0.10:
#                 anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
# #                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
#                 detects.append([int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax), row.confidence])

        #######################################################
        #                      Tracking                       #
        #######################################################
        # Update tracks using detects from current frame
        tracked_objects = tracker.update(img, frame_id, detections=detects)
        print(len(tracked_objects))
        for tobj in tracked_objects:
            bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
#             print(frame_id, last_detected_frame_id)
            if last_detected_frame_id + 1 == frame_id:  # Add only objects that were seen on the previous frame
                # Add objects that have no detections on current frame to predictions
                xc, yc = tobj.estimate[0]
                x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
                score = tobj.last_detection.scores[0]
                tracked.append([x_min, y_min, bbox_width, bbox_height])
#                 anno += '{} {} {} {} {} '.format(score, x_min, y_min, bbox_width, bbox_height)

    #             predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        #######################################################
                print('tracked!!')
                compare_img(img, tracked_img, preds, tracked, bbox_format='coco')
#     print(preds, tracked)
    if idx>0:
        break
    frame_id += 1

YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/677 [00:00<?, ?it/s]

0
1


#### import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [18]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [19]:
# model = torch.hub.load('../input/yolov5-lib-ds', 
#                        'custom', 
#                        path=CKPT_PATH,
#                        source='local',
#                        force_reload=True)  # local repo
# model.conf = 0.20
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)

YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 


In [20]:
#######################################################
#                      Tracking                       #
#######################################################

# Tracker will update tracks based on detections from current frame
# Matching based on euclidean distance between bbox centers of detections 
# from current frame and tracked_objects based on previous frames
# You can check it's parameters in norfair docs
# https://github.com/tryolabs/norfair/blob/master/docs/README.md
tracker = MatchingCOTSTracker(
    distance_function=euclidean_distance, 
    distance_threshold=20,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

# Save frame_id into detection to know which tracks have no detections on current frame
frame_id = 0
#######################################################

for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    height, width = img.shape[0], img.shape[1]
    r = model(img, size=IMG_SIZE, augment=True)
    detects = []
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > 0.01:
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
#                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
                detects.append([int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax), row.confidence])

    #######################################################
    #                      Tracking                       #
    #######################################################
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(img, frame_id, detections=detects)
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id + 1 == frame_id:  # Add only objects that were seen on the previous frame
            # Add objects that have no detections on current frame to predictions
            xc, yc = tobj.estimate[0]
            x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
            score = tobj.last_detection.scores[0]
            anno += '{} {} {} {} {} '.format(score, x_min, y_min, bbox_width, bbox_height)

#             predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    #######################################################
                
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)
    print('Prediction:', anno.strip(' '))
    frame_id += 1

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Prediction: 
Prediction: 
Prediction: 
